In [1]:
import numpy as np
import pandas as pd
import os
import csv
import seaborn as sns
import json
from decimal import Decimal
import nltk
#nltk.download()
from nltk.corpus import stopwords
import string
from pyspark.sql.functions import udf
from pyspark.sql.types import *
from afinn import Afinn
# from nltk.sentiment.vader import SentimentIntensityAnalyzer
# analyser = SentimentIntensityAnalyzer()
afinn=Afinn()
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline


from sklearn.cross_validation import train_test_split
from sklearn.metrics import classification_report

In [2]:
inputPath = "/FileStore/tables/ign"

df_data = sqlContext.read.format('csv').options(header='true', inferSchema='true').load('/FileStore/tables/ign/ign.csv')

In [3]:
display(df_data)

In [4]:
data = sqlContext.read.format("csv").option("header", "true").option("inferSchema", "true").load("/FileStore/tables/ign/ign.csv")

data.cache() # Cache data for faster reuse
data = data.dropna() # drop rows with missing values

# Register table so it is accessible via SQL Context
# For Apache Spark = 2.0
data.createOrReplaceTempView("data_tbl")
#display(data)

In [5]:
data=data.select("id","score_phrase","title","url","platform","score","genre","editors_choice","release_year","release_month","release_day").rdd.toDF()
display(data)

In [6]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler

categoricalColumns = ["title","platform","genre"]
stages = [] # stages in our Pipeline
for categoricalCol in categoricalColumns:
  # Category Indexing with StringIndexer
  stringIndexer = StringIndexer(inputCol=categoricalCol, outputCol=categoricalCol+"Index")
  # Use OneHotEncoder to convert categorical variables into binary SparseVectors
  encoder = OneHotEncoder(inputCol=categoricalCol+"Index", outputCol=categoricalCol+"classVec")
  # Add stages.  These are not run here, but will run all at once later on.
  stages += [stringIndexer, encoder]

In [7]:
label_stringIdx = StringIndexer(inputCol = "editors_choice", outputCol = "label")
stages += [label_stringIdx]

In [8]:
cols = data.columns
print(cols)

In [9]:
# Transform all features into a vector using VectorAssembler
numericCols = ["id","score"]
assemblerInputs = map(lambda c: c + "classVec", categoricalColumns) + numericCols
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
stages += [assembler]

In [10]:
# Create a Pipeline.
pipeline = Pipeline(stages=stages)
# Run the feature transformations.
#  - fit() computes feature statistics as needed.
#  - transform() actually transforms the features.
pipelineModel = pipeline.fit(data)
data = pipelineModel.transform(data)

# Keep relevant columns
selectedcols = ["label", "features"] + cols
data = data.select(selectedcols)
display(data)

In [11]:
(trainingData, testData) = data.randomSplit([0.7, 0.3], seed = 100)
print trainingData.count()
print testData.count()

In [12]:
from pyspark.ml.classification import LogisticRegression

# Create initial LogisticRegression model
lr = LogisticRegression(labelCol="label", featuresCol="features", maxIter=10)

# Train model with Training Data
lrModel = lr.fit(trainingData)

In [13]:
# Make predictions on test data using the transform() method.
# LogisticRegression.transform() will only use the 'features' column.
predictions = lrModel.transform(testData)

In [14]:
predictions.printSchema()

In [15]:
selected = predictions.select("label", "prediction", "probability", "score_phrase","genre","platform")
display(selected)

In [16]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Evaluate model
evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")
evaluator.evaluate(predictions)

In [17]:
evaluator.getMetricName()

In [18]:
print lr.explainParams()

In [19]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

# Create ParamGrid for Cross Validation
paramGrid = (ParamGridBuilder()
             .addGrid(lr.regParam, [0.01, 0.5, 2.0])
             .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0])
             .addGrid(lr.maxIter, [1, 5, 10])
             .build())

In [20]:
# Create 5-fold CrossValidator
cv = CrossValidator(estimator=lr, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=5)

# Run cross validations
cvModel = cv.fit(trainingData)
# this will likely take a fair amount of time because of the amount of models that we're creating and testing

In [21]:
predictions = cvModel.transform(testData)

In [22]:
# cvModel uses the best model found from the Cross Validation
# Evaluate best model
evaluator.evaluate(predictions)

In [23]:
print 'Model Intercept: ', cvModel.bestModel.intercept

In [24]:
weights = cvModel.bestModel.weights
# on Spark 2.X weights are available as ceofficients
weights = cvModel.bestModel.coefficients
weights = map(lambda w: (float(w),), weights)  # convert numpy type to float, and to tuple
weightsDF = sqlContext.createDataFrame(weights, ["Feature Weight"])
display(weightsDF)

In [25]:
# View best model's predictions and probabilities of each prediction class
selected = predictions.select("label", "prediction", "probability", "score_phrase","genre","platform")
display(selected)

In [26]:
from pyspark.ml.classification import DecisionTreeClassifier

# Create initial Decision Tree Model
dt = DecisionTreeClassifier(labelCol="label", featuresCol="features", maxDepth=3)

# Train model with Training Data
dtModel = dt.fit(trainingData)

In [27]:
print "numNodes = ", dtModel.numNodes
print "depth = ", dtModel.depth

In [28]:
# Make predictions on test data using the Transformer.transform() method.
predictions = dtModel.transform(testData)

In [29]:
predictions.printSchema()

In [30]:
# View model's predictions and probabilities of each prediction class
selected = predictions.select("label", "prediction", "probability", "score_phrase","genre","platform","editors_choice")
display(selected)

In [31]:
#Random Forest
from pyspark.ml.classification import RandomForestClassifier

# Create an initial RandomForest model.
rf = RandomForestClassifier(labelCol="label", featuresCol="features")

# Train model with Training Data
rfModel = rf.fit(trainingData)


In [32]:
# Make predictions on test data using the Transformer.transform() method.
predictions = rfModel.transform(testData)
predictions.printSchema()

In [33]:
# View model's predictions and probabilities of each prediction class
selected = predictions.select("label", "prediction", "probability", "score_phrase","genre","platform","editors_choice")
display(selected)

In [34]:
# Evaluate using Binary clasfrom pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Evaluate model
evaluator = BinaryClassificationEvaluator()
evaluator.evaluate(predictions)

In [35]:
# Create ParamGrid for Cross Validation
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

paramGrid = (ParamGridBuilder()
             .addGrid(rf.maxDepth, [2, 4, 6])
             .addGrid(rf.maxBins, [20, 60])
             .addGrid(rf.numTrees, [5, 20])
             .build())

In [36]:
# Create ParamGrid for Cross Validation
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

paramGrid = (ParamGridBuilder()
             .addGrid(rf.maxDepth, [2, 4, 6])
             .addGrid(rf.maxBins, [20, 60])
             .addGrid(rf.numTrees, [5, 20])
             .build())

In [37]:
# Create 5-fold CrossValidator
cv = CrossValidator(estimator=rf, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=5)

# Run cross validations.  This can take about 6 minutes since it is training over 20 trees!
cvModel = cv.fit(trainingData)

In [38]:
# cvModel uses the best model found from the Cross Validation
# Evaluate best model
evaluator.evaluate(predictions)

In [39]:
# View Best model's predictions and probabilities of each prediction class
selected = predictions.select("label", "prediction", "probability", "score_phrase","genre","platform","editors_choice")
display(selected)

In [40]:
bestModel = cvModel.bestModel

In [41]:
# Generate predictions for entire dataset
finalPredictions = bestModel.transform(data)

In [42]:
# Evaluate best model
evaluator.evaluate(finalPredictions)

In [43]:
finalPredictions.createOrReplaceTempView("finalPredictions")

In [44]:
%sql
select * from finalPredictions

In [45]:
%sql
SELECT title,score_phrase, rawPrediction,release_year
FROM finalPredictions
order by score_phrase



In [47]:
%sql
select *
from ign_csv



In [48]:
%sql
select id from ign_csv
where platform = "iPad"
order by id